# Logistic Regression with daru and statsample-glm

In this notebook we'll see with some examples how the probability of a given outcome can be predicted with logistic regression using daru and statsample-glm.

In [1]:
require 'daru'
require 'statsample-glm'
require 'open-uri'

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"http://d3js.org/d3.v3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    window.dispatchEvent(event);\n\t

true

For this notebook, we will utilize [this dataset](http://www.ats.ucla.edu/stat/data/binary.csv) denoting whether students got admission for a graduate degree program depending on their GRE scores, GPA and rank of the institute they did an undergraduate degree in (ranked from 1 to 4).

It should be noted that statsample-glm does not yet support categorical data so the ranks will be treated as continuos.

In [2]:
content = open('http://www.ats.ucla.edu/stat/data/binary.csv')
File.write('binary.csv', content.read)

df = Daru::DataFrame.from_csv "binary.csv"
df.vectors = Daru::Index.new([:admit, :gpa, :gre, :rank])
df

,admit,gpa,gre,rank
0,0,3.61,380,3
1,1,3.67,660,3
2,1,4,800,1
3,1,3.19,640,4
4,0,2.93,520,4
5,1,3,760,2
6,1,2.98,560,1
7,0,3.08,400,2
8,1,3.39,540,3
9,0,3.92,700,2


Use the `Statsampel::GLM.compute` method for logisitic regression analysis.

The first method in the `compute` function is the DataFrame object, followed by the Vector that is to be the dependent variable, and then the method to be used for the link function. Can be :logit, :probit, :poisson or :normal.

The `coefficients` method calculates the coefficients of the GLM and returns them as a Hash.

In [3]:
glm = Statsample::GLM::compute df, :admit, :logistic, constant: 1
c = glm.coefficients :hash

{:gpa=>0.777013573719857, :gre=>0.0022939595044433273, :rank=>-0.5600313868499897, :constant=>-3.4495483976684773}

The logistic regression coefficients give the change in the log odds of the outcome for a one unit increase in the predictor variable.

Therefore, to interpret each of the above co-efficients:
* For every one unit change in gre, the log odds of admission (versus non-admission) increases by **0.002**.
* For a one unit increase in gpa, the log odds of being admitted to graduate school increases by **0.777**.
* For every increase in the rank number of the institute (aka decrease in quality of the institute), the log odds of being admitted to graduate school increase by **-0.56**.

Log odds become a little difficult to interpret, so we'll exponentiate each of the co-efficients so that each co-efficient can be interpreted as an odds-ratio.

In [4]:
Daru::Vector.new(c).exp # Calling `#exp` on Daru::Vector exponentiates each element of the Vector.

#<Daru::Vector:84429200 @name = nil @size = 4 >
                                    nil
                gpa   2.174967177712439
                gre  1.0022965926425997
               rank   0.571191135676971
           constant 0.03175997601913591

We can now compute the probability of gaining admission into a graduate college based on the rank of the undergraduate college, by keeping the GRE score and GPA constant.

As you can see in the result below, the `rankp` Vector shows the probability of admission based on the rank. The person from the most highly rated undergrad school (rank 1) has a probability of **0.49** of getting admitted into graduate school.

In [5]:
e = Math::E
new_data = Daru::DataFrame.new({
  gre: [df[:gre].mean]*4,
  gpa: [df[:gpa].mean]*4,
  rank: df[:rank].factors
  })

new_data[:rankp] = new_data.collect(:row) do |x|
  1 / (1 + e ** -(c[:constant]  + x[:gre] * c[:gre] + x[:gpa] * c[:gpa] + x[:rank] * c[:rank]))
end

new_data.sort! [:rank]

,gpa,gre,rank,rankp
1,3.3899000000000017,587.7,1,0.4931450619837156
3,3.3899000000000017,587.7,2,0.357219500353945
0,3.3899000000000017,587.7,3,0.240948896129993
2,3.3899000000000017,587.7,4,0.1534862275970381


To demonstrate with another example, lets create a hypothetical dataset consisting of the body weight of 20 people and whether they survived or not.

For this example we will just assume that people with less body weight have lesser chances of survival.

In [6]:
require 'distribution'

# Create a normally distributed Vector with mean 30 and standard deviation 2
rng = Distribution::Normal.rng(30,2)
body_weight = Daru::Vector.new(20.times.map { rng.call }.sort)

# Populate chances of survival, assume that people with less body weight on average
# are less likely to survive.
survive = Daru::Vector.new [0,0,0,0,0,1,0,1,0,0,1,1,0,1,1,1,0,1,1,1]

df = Daru::DataFrame.new({
  body_weight: body_weight,
  survive: survive
})

,body_weight,survive
0,26.549216068624,0
1,27.536951405895802,0
2,27.630415185983107,0
3,27.690055295887653,0
4,28.523206001468417,0
5,29.030312984729367,1
6,29.807943543035645,0
7,29.99535853392751,1
8,30.091673469038692,0
9,30.29761751710971,0


Compute the logistic regression co-efficients.

In [7]:
glm    = Statsample::GLM.compute df, :survive, :logistic, constant: 1
coeffs = glm.coefficients :hash

{:body_weight=>0.9276058710911881, :constant=>-28.082815693129945}

Based on the coefficients, we compute the predicted probabilities for each number in the Vector :body_weight and store them in another Vector called `:survive_pred`.

In [8]:
e = Math::E
df[:survive_pred] = df[:body_weight].map { |x| 1 / (1 + e ** -(coeffs[:constant] + x*coeffs[:body_weight])) }
df

,body_weight,survive,survive_pred
0,26.549216068624,0,0.030602085999618323
1,27.536951405895802,0,0.07314333669065666
2,27.630415185983107,0,0.07924276148897372
3,27.690055295887653,0,0.08337439719954785
4,28.523206001468417,0,0.164582127639518
5,29.030312984729367,1,0.23973609014799296
6,29.807943543035645,0,0.39345976375888436
7,29.99535853392751,1,0.4356230653815916
8,30.091673469038692,0,0.45770066865408976
9,30.29761751710971,0,0.5053578438667367


The above results can then be plotted using the `plot` function.

The curve looks is an ideal logit regression curve.

In [9]:
df.plot type: [:scatter,:line], x: [:body_weight]*2, y: [:survive_pred]*2 do |plot, diagram|
  plot.x_label "Body Weight"
  plot.y_label "Probability of Survival"
end

#<Nyaplot::Frame:0x8e84f60 @properties={:panes=>[#<Nyaplot::Plot:0x8ea0224 @properties={:diagrams=>[#<Nyaplot::Diagram:0x8e85b18 @properties={:type=>:scatter, :options=>{:x=>:body_weight, :y=>:survive_pred}, :data=>"17b0a257-b84b-4051-80ed-2987d958a692"}, @xrange=[26.549216068624, 33.515674562486545], @yrange=[0.030602085999618323, 0.9528678471409187]>, #<Nyaplot::Diagram:0x8e855c8 @properties={:type=>:line, :options=>{:x=>:body_weight, :y=>:survive_pred}, :data=>"17b0a257-b84b-4051-80ed-2987d958a692"}, @xrange=[26.549216068624, 33.515674562486545], @yrange=[0.030602085999618323, 0.9528678471409187]>], :options=>{:x_label=>"Body Weight", :y_label=>"Probability of Survival", :zoom=>true, :width=>700, :xrange=>[26.549216068624, 33.515674562486545], :yrange=>[0.030602085999618323, 0.9528678471409187]}}>], :data=>{"17b0a257-b84b-4051-80ed-2987d958a692"=>#<Nyaplot::DataFrame:0x8e864b4 @name="17b0a257-b84b-4051-80ed-2987d958a692", @rows=[{:body_weight=>26.549216068624, :survive=>0, :survive_pred=>0.030602085999618323}, {:body_weight=>27.536951405895802, :survive=>0, :survive_pred=>0.07314333669065666}, {:body_weight=>27.630415185983107, :survive=>0, :survive_pred=>0.07924276148897372}, {:body_weight=>27.690055295887653, :survive=>0, :survive_pred=>0.08337439719954785}, {:body_weight=>28.523206001468417, :survive=>0, :survive_pred=>0.164582127639518}, {:body_weight=>29.030312984729367, :survive=>1, :survive_pred=>0.23973609014799296}, {:body_weight=>29.807943543035645, :survive=>0, :survive_pred=>0.39345976375888436}, {:body_weight=>29.99535853392751, :survive=>1, :survive_pred=>0.4356230653815916}, {:body_weight=>30.091673469038692, :survive=>0, :survive_pred=>0.45770066865408976}, {:body_weight=>30.29761751710971, :survive=>0, :survive_pred=>0.5053578438667367}, {:body_weight=>30.50341566956734, :survive=>1, :survive_pred=>0.5528844015750001}, {:body_weight=>30.514699417152965, :survive=>1, :survive_pred=>0.555470390835559}, {:body_weight=>30.571179195500545, :survive=>0, :survive_pred=>0.5683665015249316}, {:body_weight=>31.015315802803272, :survive=>1, :survive_pred=>0.6653378272920787}, {:body_weight=>31.557123074483492, :survive=>1, :survive_pred=>0.7666975913077382}, {:body_weight=>31.851186176960134, :survive=>1, :survive_pred=>0.8119195719539583}, {:body_weight=>32.20991302793521, :survive=>0, :survive_pred=>0.8575744681416743}, {:body_weight=>32.25127969292785, :survive=>1, :survive_pred=>0.862197243142135}, {:body_weight=>33.31693339495288, :survive=>1, :survive_pred=>0.9438620162584023}, {:body_weight=>33.515674562486545, :survive=>1, :survive_pred=>0.9528678471409187}]>}, :extension=>[]}>